# IPCC SR1.5 scenario database  
## IAMCdata package tutorial and diagnostics

<img style="float: right; height: 100px; margin-top: 10px;" src="_static/IIASA_logo.png">
<img style="float: right; height: 80px;" src="_static/IAMC_logo.jpg">

This notebook contains a number of diagnostic assessments of the scenarios submitted   
to the scenario database for the IPCC's _"Special Report on Global Warming of 1.5°C"_.  
The scenario database can be accessed at https://db1.ene.iiasa.ac.at/IPCCSR15DB.

## Load utilities package and import data from a csv snapshot file

The snapshot file can be downloaded from https://db1.ene.iiasa.ac.at/IPCCSR15DB/.  
Please contact the database admin at <ipccsr15db.ene.admin@iiasa.ac.at> for getting an account.

In [1]:
from utils import IAMCdata

<IPython.core.display.Javascript object>

In [2]:
sr1p5 = IAMCdata('ipccsr15_dbsnapshot', 'csv', ['World'])

## Display list of models and scenarios

This section illustrates some basic functionality of the ``IAMCdata`` package to explore the IPCC SR 1.5 scenario data.
1. show all models that submitted SSP scenarios 
(scenario names start with 'SSP').  
2. show all scenarios submitted by the AIM model under the ADVANCE project 
(scenario names start with 'ADV').  
3. show all timeseries identifiers (variables) related to sub-categories of ``Primary Energy|Gas`` 
reported by AIM in the ADANCE scenarios.
4. show a color-coded Pivot table of submitted models and scenarios for SSP5,  
highlighting all model/scenario combinations that have at least one timeseries.

### Note on data filtering

The feature for filtering by **model and scenario** are implemented
using [regular expressions](https://docs.python.org/3.6/library/re.html) (regex, re)
and the _re.match_ function.
This implies that the filtering is done from the start of the name.
> Filtering for _AIM_ will return all AIM models versions.
> Filtering for _IM_ will return an empty set.

Filtering for **timeseries indicators (variables)** using regex
is problematic because of the use of the `|` character. 
This package implements a pseudo-regex syntax, 
where `|` is escaped, `*` is used as a wildcard
and exact matching at the end of the string is enforced.
(in regex lingo, `*` is replaced by `.*` and `$` is appended to the filter string).
> Filtering for _Primary Energy|Gas_ will return only exactly those data.  
> Filtering for _Primary Energy|Gas|*_ will return all sub-categories of 
> natural gas at the primary-energy level (and only the sub-categories).

Filtering for **years** can be done by integer number, a list of integers, or the Python class 'range'.
Note that the last year of a range is not included, so ``range(2010,2015)`` is interpreted 
as ``[2010, 2011, 2012, 2013, 2014]``.


In [3]:
sr1p5.models({'scenario': 'SSP'})

['AIM 2.0',
 'GCAM 4.2',
 'IMAGE 2.4',
 'MESSAGE-GLOBIOM 1.0',
 'REMIND-MAgPIE 1.5',
 'WITCH-GLOBIOM 2014']

In [4]:
sr1p5.scenarios({'model': 'AIM', 'scenario': 'ADV'})

['ADVANCE_2020_1.5C-2100',
 'ADVANCE_2020_Med2C',
 'ADVANCE_2020_WB2C',
 'ADVANCE_2030_Med2C',
 'ADVANCE_2030_Price1.5C',
 'ADVANCE_2030_WB2C',
 'ADVANCE_INDC',
 'ADVANCE_NoPolicy',
 'ADVANCE_Reference']

In [5]:
sr1p5.identifiers({'model': 'AIM', 'scenario': 'ADV', 'identifier': 'Primary Energy|Gas|*'})

['Primary Energy|Gas|w/ CCS', 'Primary Energy|Gas|w/o CCS']

In [6]:
sr1p5.pivot_table(['model'], ['scenario'], {'scenario': 'SSP5'})

## Counting the number of data points and aggregating data

This section illustrates the features to count or aggregate data points for a model-scenario combination.  
Such features are helpful when identifying reporting errors or missing data by specific models.

The first example counts the number of timeseries values submitted in the category ``Primary Energy|Biomass`` 
and sub-categories (indicated by ``*``) for the year 2010 in the ``SSP2-Baseline`` scenario.
Those values that are not at the maximum (indicating non-reported/missing data) are highlighted in yellow by default.

The second example returns the mean of the same timeseries data over the entire time period 
and applying a heatmap formatting to illustrate the differences across models.

In [7]:
sr1p5.pivot_table(['region', 'identifier'], ['model'], 
                  {'scenario': 'SSP2-Baseline', 'identifier': 'Primary Energy|Biomass*', 'year': 2010}, 
                  aggregated='year')

In [8]:
sr1p5.pivot_table(['region', 'identifier', 'unit'], ['model'], 
                  {'scenario': 'SSP2-Baseline', 'identifier': 'Primary Energy|Biomass*'}, 
                  aggregated='value', function='mean', style='heat_map')

## Sanity checks and data consistency validation

As a next step, we illustrate a feature to validate model data and perform automated "sanity checks".  
This is helpful to identify reporting issues and flag 'outlier' models.

In the first example, we test whether any model/scenario has a value for ``Primary Energy|Biomass`` higher than 60 EJ in the base year 2010.  
In the second example, we apply a smaller validity range of 46 and 55 EJ/y, but only test for scenarios of the ``SSP2`` family.

In [9]:
sr1p5.validate({'Primary Energy|Biomass': {'up': 60, 'year': 2010}})

All models and scenarios satisfy the criteria


In [10]:
sr1p5.validate({'Primary Energy|Biomass': {'up': 55, 'lo': 46, 'year': 2010}}, filters={'scenario': 'SSP2'})

These model/scenarios do not satisfy the criteria:


## Categorization of scenarios

The next section will apply the categorization of scenario by used-defined specificiations.

The first example shows a criteria so stringent that no model satisfies it, namely that the exceedance probability of warming of 1.5°C is below 33% throughout the century.
The second example shows all scenarios that have an exceedance probability of warming of 1.5°C below 33% at the end of the century; these are the "well-below 1.5°C" scenarios.

The last box shows a pivot table of all scenarios categorized as 'WB1.5C'.

In [11]:
sr1p5.category('testing', 
               {'AR5 climate diagnostics|Temperature|Exceedance Probability|1.5 degC|MAGICC6': {'up': 0.33}}, display='list')

No scenario satisfies the criteria


In [12]:
sr1p5.category('WB1.5C', 
               {'AR5 climate diagnostics|Temperature|Exceedance Probability|1.5 degC|MAGICC6': {'up': 0.33, 'year': 2100}},
               display='list')

The following scenarios are categorized as 'WB1.5C':


Empty DataFrame
Columns: []
Index: [(AIM 2.0, ADVANCE_2020_1.5C-2100), (AIM 2.0, SSP1-19), (AIM 2.0, SSP2-19), (GCAM 4.2, SSP1-19), (GCAM 4.2, SSP2-19), (GCAM 4.2, SSP5-19), (IMAGE 2.4, SSP1-19), (MESSAGE-GLOBIOM 1.0, ADVANCE_2020_1.5C-2100), (MESSAGE-GLOBIOM 1.0, SSP1-19), (MESSAGE-GLOBIOM 1.0, SSP2-19), (POLES ADVANCE, ADVANCE_2020_1.5C-2100), (POLES ADVANCE, ADVANCE_2030_1.5C-2100), (POLES ADVANCE, ADVANCE_2030_Price1.5C), (POLES CDL, NPi2020_400_V3), (REMIND 1.5, Def_100$), (REMIND 1.5, NucPO_100$), (REMIND 1.5, lowEI_100$), (REMIND-MAgPIE 1.5, SSP1-19), (REMIND-MAgPIE 1.5, SSP2-19), (WITCH-GLOBIOM 2014, SSP1-19), (WITCH-GLOBIOM 2014, SSP2-19), (WITCH-GLOBIOM 2014, SSP4-19), (WITCH2016, ADVANCE_2020_1.5C-2100)]

In [13]:
sr1p5.category('WB1.5C', display='pivot')